# Predict all cause 30-day hospital readmission risk

### Importing the required libraries

In [1]:
!pip install pyarrow

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Importing libs into the python environment. These functions will be referenced later in the notebook code.

from __future__ import print_function
import os
import pandas as pd
import itertools
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import s3fs
import pyarrow.parquet as pq
from pyarrow.filesystem import S3FSWrapper


from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

# Put this when it's called
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report


# Create table for missing data analysis
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data*100

# Plot learning curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Validation score")
    plt.legend(loc="best")
    return plt


# Plot validation curve
def plot_validation_curve(estimator, title, X, y, param_name, param_range, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    train_scores, test_scores = validation_curve(estimator, X, y, param_name, param_range, cv)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    plt.plot(param_range, train_mean, color='r', marker='o', markersize=5, label='Training score')
    plt.fill_between(param_range, train_mean + train_std, train_mean - train_std, alpha=0.15, color='r')
    plt.plot(param_range, test_mean, color='g', linestyle='--', marker='s', markersize=5, label='Validation score')
    plt.fill_between(param_range, test_mean + test_std, test_mean - test_std, alpha=0.15, color='g')
    plt.grid() 
    plt.xscale('log')
    plt.legend(loc='best') 
    plt.xlabel('Parameter') 
    plt.ylabel('Score') 
    plt.ylim(ylim)
    
    
# Read single parquet file from S3
def pd_read_s3_parquet(key, bucket, s3_client=None, **args):
    if s3_client is None:
        s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    pf = ParquetFile('myfile.parq')
    return pd.read_parquet(io.BytesIO(obj['Body'].read()), **args)

# Read multiple parquets from a folder on S3 generated by spark
def pd_read_s3_multiple_parquets(filepath, bucket, s3=None, 
                                 s3_client=None, verbose=False, **args):
    if not filepath.endswith('/'):
        filepath = filepath + '/'  # Add '/' to the end
    if s3_client is None:
        s3_client = boto3.client('s3')
    if s3 is None:
        s3 = boto3.resource('s3')
    s3_keys = [item.key for item in s3.Bucket(bucket).objects.filter(Prefix=filepath)
               if item.key.endswith('.parquet')]
    if not s3_keys:
        print('No parquet found in', bucket, filepath)
    elif verbose:
        print('Load parquets:')
        for p in s3_keys: 
            print(p)
    dfs = [pd_read_s3_parquet(key, bucket=bucket, s3_client=s3_client, **args) 
           for key in s3_keys]
    return pd.concat(dfs, ignore_index=True)

# Thinking about Data

It is important to understand the relationship between different tables and the data in those tables. This is important to identify the information which is relevant to the prediction. Here is the schema.

## Cleaning and Visualizing Your Data

<img src="EHR.png">

In [3]:
import boto3
import botocore 
import io
bucket = 'readmission-data' # Update this to the bucket that was created in your lab account as part of this enviroment.
prefix = "batch-transform/output"
 
s3 = boto3.resource('s3')

Raw data files are in an S3 bucket in your AWS lab account. Twelve tables will be used (allergies, careplans, conditions,encounters, imaging-studies,immunizations, medications,observations, organizations, procedures, providers). Raw comma-separated value files will be downloaded into your Amazon Sagemaker instance, and imported into a DataFrame, where it's easier to work with the structured data. Raw files do not contain row headers, and thus labels are being assiged at import.

In [4]:
##df_first_merge = pd_read_s3_multiple_parquets(prefix, bucket)
import s3fs
import pyarrow.parquet as pq
from pyarrow.filesystem import S3FSWrapper

fs = s3fs.S3FileSystem()


# Python 3.6 or later
p_dataset = pq.ParquetDataset(
    f"s3://{bucket}/{prefix}",
    filesystem=fs
)
df_first_merge = p_dataset.read().to_pandas()

df_first_merge.head()


,patient_healthcare_expenses,encounters_encounterclass,encounters_id,patient_gender,patient_healthcare_coverage,patient_marital,encounters_total_claim_cost,patient_ethnicity,patient_id,encounters_code,...,encounters_reasoncode,procedures_code,procedures_base_cost,medications_base_cost,medications_totalcost,organizations_id,organizations_revenue,providers_speciality,providers_state,providers_utilization
0,17417.72,outpatient,63f652bc-bd5a-4efd-9894-66f760a89789,F,1141.80,M,129.16,american,94a52e2c-c2f1-4c64-a7b5-b601e2294949,698314001,...,0,NaN,NaN,36.47,437.64,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
1,1378792.06,ambulatory,747a4ad4-c6b9-483b-bd44-f8a40f7fc224,F,14478.94,M,129.16,west_indian,80ca2ea1-a3b5-4f99-9d4c-d00bb66ddf7e,185347001,...,363406005,703423002.0,17366.59,NaN,NaN,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
2,321771.74,outpatient,ba5ecdc6-86d1-4772-a9fc-1355644233e1,M,35745.71,M,129.16,german,34d1f315-2f26-43bf-b773-ad2b75c3d4b8,185349003,...,0,NaN,NaN,NaN,NaN,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
3,1324925.75,ambulatory,f721e9fc-6584-4b71-b137-29d7fca740dd,M,4326.08,M,129.16,english,b3ac5384-e415-4015-83ec-96cf73b3f8a2,185347001,...,88805009,NaN,516.65,263.49,3161.88,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
4,1324925.75,ambulatory,f721e9fc-6584-4b71-b137-29d7fca740dd,M,4326.08,M,129.16,english,b3ac5384-e415-4015-83ec-96cf73b3f8a2,185347001,...,88805009,NaN,516.65,263.49,2634.90,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0


Assumption: Taking the data in which encounter class is inpatient. For all other encounter class, patient is assumed to not be admitted in the hospital.
Data preparation for cleaning and augmentation.
Identifying the duration of Patient Admission

In [5]:
draw_missing_data_table(df_first_merge)


,Total,Percent
medications_totalcost,26373800,88.698834
medications_base_cost,26373800,88.698834
procedures_code,14475300,48.682489
procedures_base_cost,14446100,48.584285
providers_state,3300,0.011098
providers_utilization,3300,0.011098
organizations_id,3300,0.011098
organizations_revenue,3300,0.011098
providers_speciality,3300,0.011098
patient_id,0,0.000000


In [6]:
## Selecting data - Ignoring wellness (since patient not admitted)
df_first_merge = df_first_merge[df_first_merge['encounters_encounterclass'].isin(['inpatient','ambulatory','urgentcare','outpatient','emergency'])] 
df_first_merge["encounters_stop"] = pd.to_datetime(df_first_merge["encounters_stop"]).dt.tz_localize(None)
df_first_merge["encounters_start"] = pd.to_datetime(df_first_merge["encounters_start"]).dt.tz_localize(None)
df_first_merge.head()

,patient_healthcare_expenses,encounters_encounterclass,encounters_id,patient_gender,patient_healthcare_coverage,patient_marital,encounters_total_claim_cost,patient_ethnicity,patient_id,encounters_code,...,encounters_reasoncode,procedures_code,procedures_base_cost,medications_base_cost,medications_totalcost,organizations_id,organizations_revenue,providers_speciality,providers_state,providers_utilization
0,17417.72,outpatient,63f652bc-bd5a-4efd-9894-66f760a89789,F,1141.80,M,129.16,american,94a52e2c-c2f1-4c64-a7b5-b601e2294949,698314001,...,0,NaN,NaN,36.47,437.64,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
1,1378792.06,ambulatory,747a4ad4-c6b9-483b-bd44-f8a40f7fc224,F,14478.94,M,129.16,west_indian,80ca2ea1-a3b5-4f99-9d4c-d00bb66ddf7e,185347001,...,363406005,703423002.0,17366.59,NaN,NaN,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
2,321771.74,outpatient,ba5ecdc6-86d1-4772-a9fc-1355644233e1,M,35745.71,M,129.16,german,34d1f315-2f26-43bf-b773-ad2b75c3d4b8,185349003,...,0,NaN,NaN,NaN,NaN,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
3,1324925.75,ambulatory,f721e9fc-6584-4b71-b137-29d7fca740dd,M,4326.08,M,129.16,english,b3ac5384-e415-4015-83ec-96cf73b3f8a2,185347001,...,88805009,NaN,516.65,263.49,3161.88,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
4,1324925.75,ambulatory,f721e9fc-6584-4b71-b137-29d7fca740dd,M,4326.08,M,129.16,english,b3ac5384-e415-4015-83ec-96cf73b3f8a2,185347001,...,88805009,NaN,516.65,263.49,2634.90,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0


Merging Immuninzations and Encounters to single data set based on Encounter ID since there are encounters for which there was no immunization administered. Left Join to take existing data merged with Immunization data. 

Identify missing values in the data set. 

In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
df_first_merge.head(5)


,patient_healthcare_expenses,encounters_encounterclass,encounters_id,patient_gender,patient_healthcare_coverage,patient_marital,encounters_total_claim_cost,patient_ethnicity,patient_id,encounters_code,encounters_payer_coverage,encounters_stop,encounters_base_encounter_cost,patient_race,patient_birthdate,encounters_start,encounters_payer,encounters_reasoncode,procedures_code,procedures_base_cost,medications_base_cost,medications_totalcost,organizations_id,organizations_revenue,providers_speciality,providers_state,providers_utilization
0,17417.72,outpatient,63f652bc-bd5a-4efd-9894-66f760a89789,F,1141.80,M,129.16,american,94a52e2c-c2f1-4c64-a7b5-b601e2294949,698314001,0.00,2010-08-04 07:05:57,129.16,white,1983-06-15,2010-08-04 06:50:57,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,0,NaN,NaN,36.47,437.64,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
1,1378792.06,ambulatory,747a4ad4-c6b9-483b-bd44-f8a40f7fc224,F,14478.94,M,129.16,west_indian,80ca2ea1-a3b5-4f99-9d4c-d00bb66ddf7e,185347001,64.16,2013-09-29 20:09:12,129.16,black,1952-04-04,2013-09-29 19:39:12,4d71f845-a6a9-3c39-b242-14d25ef86a8d,363406005,703423002.0,17366.59,NaN,NaN,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
2,321771.74,outpatient,ba5ecdc6-86d1-4772-a9fc-1355644233e1,M,35745.71,M,129.16,german,34d1f315-2f26-43bf-b773-ad2b75c3d4b8,185349003,0.00,2015-06-30 09:33:32,129.16,white,1937-07-20,2015-06-30 09:18:32,b3221cfc-24fb-339e-823d-bc4136cbc4ed,0,NaN,NaN,NaN,NaN,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
3,1324925.75,ambulatory,f721e9fc-6584-4b71-b137-29d7fca740dd,M,4326.08,M,129.16,english,b3ac5384-e415-4015-83ec-96cf73b3f8a2,185347001,49.16,2000-11-20 22:43:39,129.16,white,1936-11-03,2000-11-20 21:58:39,d47b3510-2895-3b70-9897-342d681c769d,88805009,NaN,516.65,263.49,3161.88,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0
4,1324925.75,ambulatory,f721e9fc-6584-4b71-b137-29d7fca740dd,M,4326.08,M,129.16,english,b3ac5384-e415-4015-83ec-96cf73b3f8a2,185347001,49.16,2000-11-20 22:43:39,129.16,white,1936-11-03,2000-11-20 21:58:39,d47b3510-2895-3b70-9897-342d681c769d,88805009,NaN,516.65,263.49,2634.90,03347b4d-994e-302b-848b-58019a4e274d,184569.64,M,MA,1456.0


In [8]:
draw_missing_data_table(df_first_merge)

,Total,Percent
medications_totalcost,26373800,88.698834
medications_base_cost,26373800,88.698834
procedures_code,14475300,48.682489
procedures_base_cost,14446100,48.584285
providers_state,3300,0.011098
providers_utilization,3300,0.011098
organizations_id,3300,0.011098
organizations_revenue,3300,0.011098
providers_speciality,3300,0.011098
patient_id,0,0.000000


In [9]:
# Fill missing values
## Imputation of missing data 
##marital_status = 'NM'
##df_first_merge['patient_marital'] = df_first_merge['patient_marital'].fillna(marital_status, inplace=False)

## There are entries in which procedures_code is missing, reason could be that no procedures were performed 
## so setting the value as zero for those outliers 
df_first_merge['procedures_code'] = df_first_merge['procedures_code'].fillna(0, inplace=False)

## Setting the procedures base cost to zero since there was no procedure performed and no charge shown
df_first_merge['procedures_base_cost'] = df_first_merge['procedures_base_cost'].fillna(0, inplace=False) 

df_first_merge['providers_utilization'] = df_first_merge['procedures_base_cost'].fillna(0, inplace=False)
df_first_merge['organizations_revenue'] = df_first_merge['procedures_base_cost'].fillna(0, inplace=False)

## Due to low missing values for Patient ZIP, deleting those rows
draw_missing_data_table(df_first_merge)



,Total,Percent
medications_totalcost,26373800,88.698834
medications_base_cost,26373800,88.698834
providers_speciality,3300,0.011098
organizations_id,3300,0.011098
providers_state,3300,0.011098
providers_utilization,0,0.000000
encounters_payer_coverage,0,0.000000
encounters_encounterclass,0,0.000000
encounters_id,0,0.000000
patient_gender,0,0.000000


In [10]:
df_first_merge['patient_birthdate']= pd.to_datetime(df_first_merge['patient_birthdate'])

In [11]:
from datetime import datetime
from datetime import date
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
## Convert BIRTH DATE INTO AGE factor
df_first_merge['age'] = df_first_merge['patient_birthdate'].apply(calculate_age)
draw_missing_data_table(df_first_merge)


,Total,Percent
medications_totalcost,26373800,88.698834
medications_base_cost,26373800,88.698834
providers_state,3300,0.011098
providers_speciality,3300,0.011098
organizations_id,3300,0.011098
age,0,0.000000
encounters_payer_coverage,0,0.000000
encounters_encounterclass,0,0.000000
encounters_id,0,0.000000
patient_gender,0,0.000000


In [12]:
## Dropping the columns which are not required to do feature selection
df_first_merge=df_first_merge.drop(columns=['medications_totalcost','medications_base_cost',
                                          'patient_birthdate','encounters_id','patient_id','organizations_id',
                                         'encounters_start','encounters_stop',
                                            'encounters_payer'])

## Dropping READMISSION_HRS since it affects the model negatively and takes away the importance of all other features
#df_ft_select=df_ft_select.drop(columns=['readmission_hrs'])
df_first_merge.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297341 entries, 0 to 297340
Data columns (total 19 columns):
patient_healthcare_expenses       297341 non-null float64
encounters_encounterclass         297341 non-null object
patient_gender                    297341 non-null object
patient_healthcare_coverage       297341 non-null float64
patient_marital                   297341 non-null object
encounters_total_claim_cost       297341 non-null float64
patient_ethnicity                 297341 non-null object
encounters_code                   297341 non-null int64
encounters_payer_coverage         297341 non-null float64
encounters_base_encounter_cost    297341 non-null float64
patient_race                      297341 non-null object
encounters_reasoncode             297341 non-null int64
procedures_code                   297341 non-null float64
procedures_base_cost              297341 non-null float64
organizations_revenue             297341 non-null float64
providers_speciality        

In [13]:
## Identifying categorical variables in the data set i.e. enumerated values for variables such as Gender, Organization State,Encounter Class, Encounter Code, Encounter Cost, Marital Status
## We are identifying unique values for a column, if unique values are less than 25 we consider it as categorical variable
df_first_merge['encounters_encounterclass'] = pd.Categorical(df_first_merge['encounters_encounterclass'])
df_first_merge['patient_gender'] = pd.Categorical(df_first_merge['patient_gender'])
df_first_merge['patient_marital'] = pd.Categorical(df_first_merge['patient_marital'])
df_first_merge['patient_ethnicity'] = pd.Categorical(df_first_merge['patient_ethnicity'])
df_first_merge['patient_race'] = pd.Categorical(df_first_merge['patient_race'])
df_first_merge['providers_speciality'] = pd.Categorical(df_first_merge['providers_speciality'])
df_first_merge['providers_state'] = pd.Categorical(df_first_merge['providers_state'])
df_first_merge['encounters_reasoncode'] = pd.Categorical(df_first_merge['encounters_reasoncode'])
df_first_merge['encounters_code'] = pd.Categorical(df_first_merge['encounters_code'])
df_first_merge['procedures_code'] = pd.Categorical(df_first_merge['procedures_code'])

# Transform categorical variables into dummy variables
df_first_merge = pd.get_dummies(df_first_merge, drop_first=True)  # To avoid dummy trap




The resulted DataFrame is serialized and written to a flat file called the Pickle using the **Pickle** library. This file is then saved into the Amazon S3 bucket for later re-use of the data. You can generate a Pickle file using `pickle.dump` and save the raw datafile in that object and upload the file to the lab S3 bucket.

In [14]:
## As a next step, we can use trusted pickle to check for signature on pickle dump when it is read to avoid spoofing
import pickle
with open('readmission-predict-input-data.pkl', 'wb') as handle:
    pickle.dump(df_first_merge, handle, protocol=pickle.HIGHEST_PROTOCOL)
s3.Bucket(bucket).upload_file('readmission-predict-input-data.pkl','batch-transform/processed-data/readmission-predict-input-data.pkl',ExtraArgs={"ServerSideEncryption": "aws:kms","SSEKMSKeyId":"3a90a5d2-2ba8-4942-b9df-9a27ff7bf412" })
